# Day3課題 - 鈴木雅彦

## 前置き

11/3(土)のDay3講義中に以下のことが決まった。
- 全員で精度を競いつつ、有効な手法を共有する
- 対象の課題は「Car Fuel Consumption」（回帰問題）で統一する
- 先頭10%をテストデータ、残りの90%を学習用データとする

鈴木はDay1,2課題にて「Kickstarter Projects」（分類問題）に取り組んでいたので１から取り組む。

## Pythonライブラリを読み込む

In [1]:
# データハンドリング
import numpy as np
import pandas as pd

# 可視化
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# 前処理
from sklearn.preprocessing import MinMaxScaler # 正規化
from sklearn.preprocessing import StandardScaler # 標準化

# 汎化誤差検証
from sklearn.model_selection import train_test_split # ホールドアウト法
from sklearn.model_selection import KFold # 交差検証法

# 分類問題
from sklearn.linear_model import SGDClassifier # ロジスティック回帰
from sklearn.metrics import confusion_matrix # 結果の可視化
from sklearn.metrics import log_loss # 性能評価
from sklearn.metrics import accuracy_score # 性能評価
from sklearn.metrics import precision_score # 性能評価
from sklearn.metrics import recall_score # 性能評価

# 回帰問題


## データファイルを取得する

1. [Car Fuel Consumption | Kaggle](https://www.kaggle.com/anderas/car-consume)ページを開く

1. [DownloadAll](https://www.kaggle.com/anderas/car-consume/downloads/car-consume.zip/5) リンクからZIPをダウンロードする

1. ZIPを解答する
```
$ cd ~/Downloads
$ unzip car-consume.zip
```

1. データファイルを作業フォルダに配置する
```
$ cp ~/Downloads/measurements.csv ~/Desktop/ml_tokyo_8/git/data/
```

1. gitにpushしておく
```
$ git add data/measurements.csv 
$ git commit -m 'https://www.kaggle.com/anderas/car-consume'
$ git push
```

|型 |名称 |説明 |
|---|-----|----|
|# |distance        |距離 |
|# |consume       |消費量 |
|# |speed            |速度 |
|# |temp_inside   |車内温度 |
|# |temp_outside |社外温度 |
|A |specials         |特記事項 (ACとrainの内容と重複) |
|A |gas_type       |ガソリン種別 SP98:ハイオク, E10:バイオエタノール10％混合 |
|# |AC                |AC電源利用有無(？) |
|# |rain               |雨が降っていたかどうか |
|# |sun               | |
|# |refill liters      |給油量 |
|A |refill gas        |給油ガソリン種別 |

In [2]:
# ファイルの確認
!pwd
!wc data/measurements.csv

/Users/suzukimasahiko/Desktop/ml_tokyo_8/git
     389     410   14561 data/measurements.csv


## データファイルの読み込む

In [52]:
df = pd.read_csv('data/measurements.csv')

# データフレームの情報を表示する関数
# この後も何度か使うので関数として定義しておく
def show_dataframe_info(df):
    display(df.shape)
    display(df.index)
    display(pd.DataFrame({
        'name': df.columns,
        'dtypes': df.dtypes,
        'count': df.count(),
        'null': df.isnull().sum()
    }))
    df.head()

show_dataframe_info(df)

(388, 12)

RangeIndex(start=0, stop=388, step=1)

,name,dtypes,count,null
distance,distance,object,388,0
consume,consume,object,388,0
speed,speed,int64,388,0
temp_inside,temp_inside,object,376,12
temp_outside,temp_outside,int64,388,0
specials,specials,object,93,295
gas_type,gas_type,object,388,0
AC,AC,int64,388,0
rain,rain,int64,388,0
sun,sun,int64,388,0


## 異常値と欠損値を補正する

In [53]:
# 数値の表記がおかしいので修正する

# 文字列型変更 → カンマをドットに置換 → フロート型に変換
f_comma2dot = lambda x: str(x).replace(',', '.')
for column in ['distance', 'consume', 'speed', 'temp_inside', 'temp_outside', 'AC', 'rain', 'sun', 'refill liters']:
    df[column] = df[column].apply(f_comma2dot).astype(float)

show_dataframe_info(df)

(388, 12)

RangeIndex(start=0, stop=388, step=1)

,name,dtypes,count,null
distance,distance,float64,388,0
consume,consume,float64,388,0
speed,speed,float64,388,0
temp_inside,temp_inside,float64,376,12
temp_outside,temp_outside,float64,388,0
specials,specials,object,93,295
gas_type,gas_type,object,388,0
AC,AC,float64,388,0
rain,rain,float64,388,0
sun,sun,float64,388,0


## 学習データとテストデータを分割

In [54]:
# 先頭10%をテストデータ、後半90%を学習データとして扱う
# あえてsklearn.model_selection.train_test_split()は使わない
test_size = 0.1
test_row_num = int(df.shape[0] * test_size) # 端数切り捨て int(x) = int(math.floor(x))
df_test = df.iloc[:test_row_num]
df_train = df.iloc[test_row_num:]